# [Module 1.2] 로컬 모드 및 스크립트 모드로 훈련 (SageMaker 사용)

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 기본 환경 세팅
- 데이터 세트를 S3에 업로드
- 스크립트 모드의 코드 작성 방법
- 훈련 코드 확인
- 로컬 모드로 훈련 실행
- SageMaker Host Mode 로 훈련

---

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [2]:
### 커스텀 라이브러리
import config 

In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

use_default_bucket = True
if use_default_bucket:
    bucket = sagemaker_session.default_bucket()
else:
    bucket = '<Type your bucket>'
    
prefix = "NCFModel"



#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [4]:
import os
import subprocess


try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
    else:
        instance_type = "local"        
except:
    pass

print("Instance type = " + instance_type)

Instance type = local_gpu


## 스크립트 모드의 코드 작성 방법
- ![script_mode_example](img/script_mode_example.png)

## 훈련 코드 확인
- 아래의 코드는 전형적인 스크립트 모드의 코드 작성 방법을 따르고 있습니다.
- 훈련 함수는 `from train_lib import train` 로서 이전 노트북의 **[세이지 메이커 없이]** 작성한 스크래치 버전에서 사용한 훈련 함수와 동일 합니다.


In [5]:
train_code = 'src/train.py'
!pygmentize {train_code}

import argparse
import os
import json
import sys
sys.path.append('./src')

from train_lib import train

if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    ##################################
    #### 세이지 메이커 프레임워크의 도커 컨테이너 환경 변수 인자
    ##################################

    parser.add_argument('--train-data-dir', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    
    parser.add_argument('--test-data-dir', type=str, default=os.environ['SM_CHANNEL_TEST'])

    
        
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])    
           
    
    ##################################
    #### 사용자 정의 커맨드 인자
    ##################################

    parser.add_argument("--lr", 
        type=float, 
        default=0.001, 
        help="learning rate")
    parser.add_argument("--dropout", 
        type=float,
        default=0.0,  
        help="dropout rate")
    parser.add_argument("--batch_size", 
        type=int, 
        default=2

### 로컬에 있는 데이타 세트의 위치를 지정 합니다.

In [6]:
local_inputs = config.main_path
print("local_inputs: ", local_inputs)

local_inputs:  ../../NCF-Data/


### 로컬 모드로 훈련 실행
- 아래의 두 라인이 로컬모드로 훈련을 지시 합니다.
```python
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
```

In [7]:
hyperparameters = {'epochs': 1, 
                   'lr': 0.001,
                   'batch_size': 256,
                   'top_k' : 10,
                   'dropout' : 0.0,
                   'factor_num' : 32,
                   'num_layers' : 3,
                   'num_ng' : 4,
                   'test_num_ng' : 99,                   
                    }  

In [8]:
local_inputs = {'train': f'file://{local_inputs}',
          'test': f'file://{local_inputs}'}

In [9]:
from sagemaker.pytorch import PyTorch
import os
import subprocess


local_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
    hyperparameters= hyperparameters               
    
)
local_estimator.fit(local_inputs)

Creating gtp9d0f4jg-algo-1-sf7c5 ... 
Creating gtp9d0f4jg-algo-1-sf7c5 ... done
Attaching to gtp9d0f4jg-algo-1-sf7c5
gtp9d0f4jg-algo-1-sf7c5 | 2022-05-21 07:00:50,431 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
gtp9d0f4jg-algo-1-sf7c5 | 2022-05-21 07:00:50,453 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
gtp9d0f4jg-algo-1-sf7c5 | 2022-05-21 07:00:50,456 sagemaker_pytorch_container.training INFO     Invoking user training script.
gtp9d0f4jg-algo-1-sf7c5 | 2022-05-21 07:00:50,654 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
gtp9d0f4jg-algo-1-sf7c5 | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
gtp9d0f4jg-algo-1-sf7c5 | Requirement already satisfied: nvidia-ml-py3==7.352 in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (7.352.0)
gtp9d0f4jg-algo-1-sf7c5 | Collecting pandas==0.24.2
gtp9d0f4jg-algo-1-sf7c5 |   Downloading

## SageMaker Host Mode 로 훈련
- `cifar10_estimator.fit(inputs, wait=False)`
    - 입력 데이터를 inputs로서 S3 의 경로를 제공합니다.
    - wait=False 로 지정해서 async 모드로 훈련을 실행합니다. 
        - 실행 경과는 아래의 cifar10_estimator.logs() 에서 확인 합니다.

### 데이터 세트를 S3에 업로드


In [10]:
s3_data_loc = sagemaker_session.upload_data(path=config.main_path, bucket=bucket, 
                                       key_prefix=f"{prefix}/data")
print("s3_data_loc: ", s3_data_loc)

s3_data_loc:  s3://sagemaker-us-east-1-057716757052/NCFModel/data


In [11]:
! aws s3 ls {s3_data_loc} --recursive

2022-05-21 07:03:58     128039 NCFModel/data/.ipynb_checkpoints/ml-1m.test-checkpoint.rating
2022-05-21 07:03:58    2891424 NCFModel/data/ml-1m.test.negative
2022-05-21 07:03:55     128039 NCFModel/data/ml-1m.test.rating
2022-05-21 07:03:55   20982911 NCFModel/data/ml-1m.train.rating
2022-05-21 07:03:58   27404899 NCFModel/data/pinterest-20.test.negative
2022-05-21 07:03:54     807267 NCFModel/data/pinterest-20.test.rating
2022-05-21 07:03:55   21138451 NCFModel/data/pinterest-20.train.rating


In [12]:
# ! aws s3 rm {s3_data_loc} --recursive

In [13]:
s3_inputs = {'train': f'{s3_data_loc}',
          'test': f'{s3_data_loc}'}
print("s3_inputs: \n", s3_inputs)

s3_inputs: 
 {'train': 's3://sagemaker-us-east-1-057716757052/NCFModel/data', 'test': 's3://sagemaker-us-east-1-057716757052/NCFModel/data'}


In [14]:
host_hyperparameters = {'epochs': 1, 
                   'lr': 0.001,
                   'batch_size': 256,
                   'top_k' : 10,
                   'dropout' : 0.0,
                   'factor_num' : 32,
                   'num_layers' : 3,
                   'num_ng' : 4,
                   'test_num_ng' : 99,                   
                    }  

In [15]:
from sagemaker.pytorch import PyTorch

instance_type = 'ml.p3.2xlarge'

host_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type,
    session = sagemaker.Session(), # 세이지 메이커 세션
    hyperparameters=host_hyperparameters
    
)
host_estimator.fit(s3_inputs, wait=False)

In [16]:
%%time

host_estimator.logs()

2022-05-21 07:03:59 Starting - Starting the training job...
2022-05-21 07:04:28 Starting - Preparing the instances for trainingProfilerReport-1653116639: InProgress
.........
2022-05-21 07:05:51 Downloading - Downloading input data...
2022-05-21 07:06:25 Training - Downloading the training image..............................
2022-05-21 07:11:27 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-21 07:11:29,988 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-21 07:11:30,010 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-21 07:11:30,018 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-21 07:11:30,522 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip insta

## 모델 아티펙트 저장
- S3 에 저장된 모델 아티펙트를 저장하여 추론시 사용합니다.

In [17]:
artifact_path = host_estimator.model_data
print("artifact_path: ", artifact_path)

%store artifact_path

artifact_path:  s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-05-21-07-03-59-050/output/model.tar.gz
Stored 'artifact_path' (str)


In [18]:
%store bucket 
%store prefix

Stored 'bucket' (str)
Stored 'prefix' (str)
